# League of Legends ETL (Extract, Transform, Load)

* With the World Championship Series underway, I often find myself with some very specific questions about the various players, teams, and specific games. Though the data is available in CSV form through a third party organization (Shoutout to Tim Sevenhuysen at https://oracleselixir.com/), it can be hard to answer some of my questions just by looking at the Excel spreadsheet. 
<br>

* I've decided to generate an ETL process using Python and SQL server as a means of warehousing data for easy querying (and as a fun exercise). The plan is as follows:

    1. Acquire the dataset and preparing it for loading using Python
    
    2. Design a relational database for the warehouse
    
    3. Enforce specific validation rules on the data using Python
    
    4. Clean and transform based on validation rules and database requirements
    
    5. Load into a SQL Server relational database that I designed using Python and pyodbc
    
    6. Extract insights using SQL
    
    7. Hook up PowerBI to the database to generate a dashboard for further insights

In [9]:
# Housekeeping
import numpy as np
import pandas as pd
import pyodbc
import requests
import json

## The "E" in ETL

In [7]:
# Data can be retrieved with just a URL in this case
url = 'https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2020_LoL_esports_match_data_from_OraclesElixir_20201022.csv'
data = pd.read_csv(url)
data.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15
0,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,23.0,4888.0,7368.0,131.0,4723.0,7202.0,118.0,165.0,166.0,13.0
1,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,-10.0,4385.0,4817.0,91.0,4784.0,4667.0,98.0,-399.0,150.0,-7.0
2,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,-9.0,4809.0,6275.0,129.0,5218.0,8112.0,140.0,-409.0,-1837.0,-11.0
3,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,3.0,4915.0,4959.0,136.0,4864.0,5360.0,135.0,51.0,-401.0,1.0
4,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,1.0,2956.0,3800.0,20.0,3189.0,3543.0,28.0,-233.0,257.0,-8.0


## Designing the Database

* Now that we have access to the data, we need a place to put it that lets us easily query and extract information. I will be using `pyodbc` to create a new SQL Server database that we can then load the data into.

In [18]:
# Generate Connection String
config = json.loads(open('config.json', 'r').read())
driver = '{ODBC Driver 17 for SQL Server}'
conn_str = f"DRIVER={driver};SERVER={config['SERVER']};DATABASE={config['DATABASE']};UID={config['USERNAME']};PASSWORD={config['PASSWORD']};"
print(conn_str)
conn = pyodbc.connect(conn_str)

DRIVER={ODBC Driver 17 for SQL Server};SERVER=LAPTOP-0S64IKJ8;DATABASE=League;UID=worlds-etl;PASSWORD=!#Y^X6c#523XtuXSRpmu;


InterfaceError: ('28000', "[28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'worlds-etl'. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 17 for SQL Server]Invalid connection string attribute (0); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'worlds-etl'. (18456); [28000] [Microsoft][ODBC Driver 17 for SQL Server]Invalid connection string attribute (0)")

## Establish Validation Rules

* We only want to load data into the database given it's cleaned and ready, so it's important to establish rules for validating that the data is in a form we can work with. The following are some rules we need to validate first.

    - Records to be loaded cannot be in the database already
    - Records need to be complete, as established by the `datacompleteness` column
    - 